In [1]:
import instructor
from pydantic import BaseModel, Field, field_validator, ConfigDict
from openai import OpenAI
from typing import Optional, List, Iterable
from typing_extensions import Literal
import re
from dotenv import load_dotenv
import os

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [ ]:
load_dotenv()

In [ ]:
api_key = os.getenv("api_key")
client = instructor.from_openai(OpenAI(api_key))

In [2]:
from tqdm.auto import tqdm
import pandas as pd

tqdm.pandas()

In [3]:
df = pd.read_csv("combined_descriptions.csv")
df

,image,url,sections,page_path,published_at,title,updated_at,type,description,external_id,...,cdp,Color Collection,Mood,Project,Style,Opacity,Value,Color Family,cleaned_name,Tone
0,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,"['Caliente', 'Matching Colors', 'Different Sha...",/en-us/paint-colors/color/af-290/caliente,2024-07-21T23:34:03Z,Caliente AF-290 | Benjamin Moore,2024-07-21T23:34:03Z,Color,"A charismatic, classic shade of red that is ra...",87eb01f88fbf7d68caee640bdfc86a86167c6976,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Caliente,NaN
1,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,"['Dinner Party', 'Matching Colors', 'Different...",/en-us/paint-colors/color/af-300/dinner-party,2024-07-21T23:39:52Z,Dinner Party AF-300 | Benjamin Moore,2024-07-21T23:39:52Z,Color,"A deep, understated red that can be used in ev...",5dba8a688cdd0bd12c6b2c7791ee934a5a63c97d,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Dinner Party,NaN
2,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,"['New London Burgundy', 'Matching Colors', 'Di...",/en-us/paint-colors/color/hc-61/new-london-bur...,2024-07-21T23:58:04Z,New London Burgundy HC-61 | Benjamin Moore,2024-07-21T23:58:03Z,Color,"A distinguished, deep red with violet undertones.",1ed55b97c8faa1b72b5d0a6b27e23c2d2d62dcd3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New London Burgundy,NaN
3,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,"['Million Dollar Red', 'Matching Colors', 'Dif...",/en-us/paint-colors/color/2003-10/million-doll...,2024-07-21T23:48:53Z,Million Dollar Red 2003-10 | Benjamin Moore,2024-07-21T23:48:53Z,Color,A signature red for interiors that perfectly b...,6300df03dd4cecc0ff72ae7104d542d658a22607,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Million Dollar Red,NaN
4,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,"['Garrison Red', 'Matching Colors', 'Similar C...",/en-us/paint-colors/color/hc-66/garrison-red,2024-07-21T21:26:13Z,Garrison Red HC-66 | Benjamin Moore,2024-07-21T21:26:13Z,Color,Muted yet balanced undertones that go from bri...,12b374ca867d58775d24debdce036e52072670c2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Garrison Red,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3653,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,"[""Marilyn's Dress"", 'Matching Colors', 'Differ...",/en-us/paint-colors/color/2125-60/marilyns-dress,2024-07-21T21:22:05Z,Marilyn's Dress 2125-60 | Benjamin Moore,2024-07-21T21:22:05Z,Color,"Much like its namesake, this pale blue-gray is...",131ede0694dd47eaf161b4579d04e4e682b5209f,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Marilyn's Dress,NaN
3654,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,"['Tundra', 'Matching Colors', 'Different Shade...",/en-us/paint-colors/color/2133-70/tundra,2024-07-21T09:00:21Z,Tundra 2133-70 | Benjamin Moore,2024-07-21T09:00:21Z,Color,Trace amounts of gray and violet come through ...,6e037ee23fe11ec6c2979db140c6f6ef7f27a803,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Tundra,NaN
3655,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,"['Winter Snow', 'Matching Colors', 'Similar Co...",/en-us/paint-colors/color/oc-63/winter-snow,2024-07-22T01:31:20Z,Winter Snow OC-63 | Benjamin Moore,2024-07-22T01:31:20Z,Color,Clean blue undertones define this optic white.,ae5eb68474567462c0ad84186570f81e9abc7f36,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Winter Snow,NaN
3656,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,"[""Baby's Breath"", 'Matching Colors', 'Differen...",/en-us/paint-colors/color/873/babys-breath,2024-07-21T23:12:55Z,Baby's Breath 873 | Benjamin Moore,2024-07-21T23:12:55

In [4]:
class Description(BaseModel):
    is_color_description: Literal["yes", "no", "unsure"] = Field(description="Is this describing a paint color?")
    is_poetry: Literal["yes", "no", "unsure"] = Field(description="Is this poetic?")
    writing_form: Literal["poetry", "prose", "unsure"] = Field(description="Is this poetry or prose?")
    writing_type: Literal["poetry", "story", "color description", "historically specific"] = Field(description="What type of writing is this?")

In [5]:
def get_details(row):
    paint_description = row['description']

    result = client.chat.completions.create(
        model="gpt-4o-mini",
        response_model=Description,
        messages=[
            {"role": "user", "content": paint_description}
        ],
        temperature=0
    )
    
    return result.model_dump()

In [41]:
get_details(subset.iloc[0])
# test for first row

{'is_color_description': 'yes',
 'is_poetry': 'no',
 'writing_form': 'prose',
 'writing_type': 'color description'}

In [42]:
subset = ff.sample(100).reset_index(drop=True)

In [43]:
details = subset.progress_apply(get_details, axis=1)
details_df = pd.json_normalize(details)
details_df

  0%|          | 0/100 [00:00<?, ?it/s]

,is_color_description,is_poetry,writing_form,writing_type
0,yes,no,prose,color description
1,yes,no,prose,color description
2,yes,no,prose,color description
3,yes,no,prose,color description
4,yes,no,prose,color description
...,...,...,...,...
95,yes,no,prose,color description
96,no,no,prose,story
97,yes,no,prose,color description
98,yes,no,prose,color description


In [64]:
merged = subset.join(details_df)
merged

,sections,page_path,title,type,description,external_id,image,url,published_at,updated_at,...,Mood,Project,Style,Value,Color Family,Tone,is_color_description,is_poetry,writing_form,writing_type
0,"['Dusty Ranch Brown', 'Matching Colors', 'Diff...",/en-us/paint-colors/color/2105-40/dusty-ranch-...,Dusty Ranch Brown 2105-40 | Benjamin Moore,Color,A rosy brown hue deepened with a trace of viol...,529b113ef201ef797562635cd690e7b63b7ef4ec,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T04:36:14Z,2024-07-22T04:36:14Z,...,NaN,NaN,NaN,NaN,NaN,NaN,yes,no,prose,color description
1,"['Yellow Tone', 'Matching Colors', 'Different ...",/en-us/paint-colors/color/370/yellow-tone,Yellow Tone 370 | Benjamin Moore,Color,"Possessing a controlled vibrancy, this yellow-...",0e975b80a9b8bbe43e76db600e4d21345e198e4d,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-21T20:36:29Z,2024-07-21T20:36:28Z,...,NaN,NaN,NaN,NaN,NaN,NaN,yes,no,prose,color description
2,"['Royal Linen', 'Matching Colors', 'Different ...",/en-us/paint-colors/color/931/royal-linen,Royal Linen 931 | Benjamin Moore,Color,A soft yellow that echoes the organic warmth o...,4adfdaa3ed1af3a11fac1651cf10b4c985ec9d43,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T03:31:43Z,2024-07-22T03:31:42Z,...,NaN,NaN,NaN,NaN,NaN,NaN,yes,no,prose,color description
3,"['Aztec Yellow', 'Matching Colors', 'Different...",/en-us/paint-colors/color/2152-20/aztec-yellow,Aztec Yellow 2152-20 | Benjamin Moore,Color,"Exhibiting shades of amber, honey and bronze, ...",80df3c11933f73463184d919a55279a28daae469,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T03:26:22Z,2024-07-22T03:26:21Z,...,NaN,NaN,NaN,NaN,NaN,NaN,yes,no,prose,color description
4,"['Dunmore Green', 'Matching Colors', 'Color By...",/en-us/paint-colors/color/cw-540/dunmore-green,Dunmore Green CW-540 | Benjamin Moore,Color,This historic yet high-spirited green is based...,58d8057a7e9e63e9200c1d2471eb7165bdd74d39,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-03T11:20:47Z,2024-07-03T11:20:46Z,...,NaN,NaN,NaN,NaN,NaN,NaN,yes,no,prose,color description
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,"['Vernazza Yellow', 'Matching Colors', 'Differ...",/en-us/paint-colors/color/325/vernazza-yellow,Vernazza Yellow 325 | Benjamin Moore,Color,"A light, refreshing yellow that brings to mind...",9984c53179e26f041cc9afc1d72f297e9eaf7e20,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T04:11:04Z,2024-07-22T04:11:03Z,...,NaN,NaN,NaN,NaN,NaN,NaN,yes,no,prose,color description
96,"['Jungle Canopy', 'Matching Colors', 'Differen...",/en-us/paint-colors/color/csp-900/jungle-canopy,Jungle Canopy CSP-900 | Benjamin Moore,Color,"The lush, green ceiling gently shades the rain...",b376da38ec7bf2d1d9e0a4bd2e192774711fad72,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T02:19:08Z,2024-07-22T02:19:07Z,...,NaN,NaN,NaN,NaN,NaN,NaN,no,no,prose,story
97,"['Garrison Red', 'Matching Colors', 'Similar C...",/en-us/paint-colors/color/hc-66/garrison-red,Garrison Red HC-66 | Benjamin Moore,Color,Muted yet balanced undertones that go from bri...,12b374ca867d58775d24debdce036e52072670c2,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-21T21:26:13Z,2024-07-21T21:26:13Z,...,NaN,NaN,NaN,NaN,NaN,NaN,yes,no,prose,color description
98,"['Bali', 'Matching Colors', 'Different Shades'...",/en-us/paint-colors/color/702/bali,Bali 702 | Benjamin Moore,Color,A pillow-soft green dusted with pleasing gray ...,a67cf5fd33753c457aad3be657a0fd49297bb129,https://media.benjaminmoore.

In [65]:
merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 44 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   sections              100 non-null    object 
 1   page_path             100 non-null    object 
 2   title                 100 non-null    object 
 3   type                  100 non-null    object 
 4   description           100 non-null    object 
 5   external_id           100 non-null    object 
 6   image                 100 non-null    object 
 7   url                   100 non-null    object 
 8   published_at          100 non-null    object 
 9   updated_at            100 non-null    object 
 10  cdp                   11 non-null     object 
 11  popularity            100 non-null    int64  
 12  info                  0 non-null      float64
 13  _index                33 non-null     object 
 14  _type                 33 non-null     object 
 15  _score                33

In [74]:
# rerunning with the full dataset
full_details = df.progress_apply(get_details, axis=1)

  0%|          | 0/3657 [00:00<?, ?it/s]

,is_color_description,is_poetry,writing_form,writing_type
0,yes,no,prose,color description
1,yes,no,prose,color description
2,yes,no,prose,color description
3,yes,no,prose,color description
4,yes,no,prose,color description
...,...,...,...,...
95,yes,no,prose,color description
96,no,no,prose,story
97,yes,no,prose,color description
98,yes,no,prose,color description


In [80]:
full_details_df = pd.json_normalize(full_details)
full_details_df

,is_color_description,is_poetry,writing_form,writing_type
0,yes,no,prose,color description
1,yes,yes,poetry,color description
2,yes,no,prose,color description
3,yes,no,prose,color description
4,yes,no,prose,color description
...,...,...,...,...
3652,yes,no,prose,color description
3653,yes,no,prose,color description
3654,yes,no,prose,color description
3655,yes,no,prose,color description


In [81]:
full_merged = df.join(full_details_df)
full_merged
# this is the full dataset of classified descriptions

,sections,page_path,title,type,description,external_id,image,url,published_at,updated_at,...,Project,Style,Value,Color Family,Tone,cleaned_name,is_color_description,is_poetry,writing_form,writing_type
0,"['Hunter Green', 'Matching Colors', 'Different...",/en-us/paint-colors/color/2041-10/hunter-green,Hunter Green 2041-10 | Benjamin Moore,Color,A timeless dark green full of depth and elegance.,5bbc15c9d8fa62ad213913ec73d45dd3fd96927f,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T00:32:36Z,2024-07-22T00:32:36Z,...,NaN,NaN,NaN,NaN,NaN,Hunter Green,yes,no,prose,color description
1,"['October Mist', 'Matching Colors', 'Different...",/en-us/paint-colors/color/1495/october-mist,October Mist 1495 | Benjamin Moore,Color,"Evocative of the stem of a flower, this gently...",650d633a3fb633324afcf6161c43826204085af7,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-21T22:48:45Z,2024-07-21T22:48:45Z,...,NaN,NaN,NaN,NaN,NaN,October Mist,yes,yes,poetry,color description
2,"['Hollingsworth Green', 'Matching Colors', 'Di...",/en-us/paint-colors/color/hc-141/hollingsworth...,Hollingsworth Green HC-141 | Benjamin Moore,Color,"A feather-light combination of green, blue and...",c3b74b8ba97769e1a4ef88f2ec55bdeef8b02d1f,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-21T22:40:18Z,2024-07-21T22:40:18Z,...,NaN,NaN,NaN,NaN,NaN,Hollingsworth Green,yes,no,prose,color description
3,"['Guilford Green', 'Matching Colors', 'Differe...",/en-us/paint-colors/color/hc-116/guilford-green,Guilford Green HC-116 | Benjamin Moore,Color,A gentle green hue with pleasing warm undertones.,30c65a442a898b35d8c2433b1f7d565b4fcef0ec,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T00:15:41Z,2024-07-22T00:15:40Z,...,NaN,NaN,NaN,NaN,NaN,Guilford Green,yes,no,prose,color description
4,"['Soft Fern', 'Matching Colors', 'Different Sh...",/en-us/paint-colors/color/2144-40/soft-fern,Soft Fern 2144-40 | Benjamin Moore,Color,A pleasing pale green misted with gray tones.,1814c4578a537d1e3d02b407e92c0b11e06b2109,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-21T21:00:09Z,2024-07-21T21:00:08Z,...,NaN,NaN,NaN,NaN,NaN,Soft Fern,yes,no,prose,color description
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3652,"[""Marilyn's Dress"", 'Matching Colors', 'Differ...",/en-us/paint-colors/color/2125-60/marilyns-dress,Marilyn's Dress 2125-60 | Benjamin Moore,Color,"Much like its namesake, this pale blue-gray is...",131ede0694dd47eaf161b4579d04e4e682b5209f,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-21T21:22:05Z,2024-07-21T21:22:05Z,...,NaN,NaN,NaN,NaN,NaN,Marilyn's Dress,yes,no,prose,color description
3653,"['Tundra', 'Matching Colors', 'Different Shade...",/en-us/paint-colors/color/2133-70/tundra,Tundra 2133-70 | Benjamin Moore,Color,Trace amounts of gray and violet come through ...,6e037ee23fe11ec6c2979db140c6f6ef7f27a803,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-21T09:00:21Z,2024-07-21T09:00:21Z,...,NaN,NaN,NaN,NaN,NaN,Tundra,yes,no,prose,color description
3654,"['Winter Snow', 'Matching Colors', 'Similar Co...",/en-us/paint-colors/color/oc-63/winter-snow,Winter Snow OC-63 | Benjamin Moore,Color,Clean blue undertones define this optic white.,ae5eb68474567462c0ad84186570f81e9abc7f36,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T01:31:20Z,2024-07-22T01:31:20Z,...,NaN,NaN,NaN,NaN,NaN,Winter Snow,yes,no,prose,color description
3655,"[""Baby's Breath"", 'Matching Colors', 'Differen...",/en-us/paint-colors/color/873/babys-breath,Baby's Breath 873 | Benjamin Moore,Color,

In [82]:
full_merged['is_color_description'].value_counts()

is_color_description
yes    3554
no      103
Name: count, dtype: int64

In [83]:
full_details_df['is_color_description'].value_counts()

is_color_description
yes    3554
no      103
Name: count, dtype: int64

In [84]:
full_details_df['is_poetry'].value_counts()

is_poetry
no     3417
yes     240
Name: count, dtype: int64

In [85]:
full_details_df['writing_form'].value_counts()

writing_form
prose     3423
poetry     234
Name: count, dtype: int64

In [86]:
full_details_df['writing_type'].value_counts()

writing_type
color description        3553
story                      69
poetry                     34
historically specific       1
Name: count, dtype: int64

In [87]:
full_merged[full_merged['writing_form'] == 'poetry']

,sections,page_path,title,type,description,external_id,image,url,published_at,updated_at,...,Project,Style,Value,Color Family,Tone,cleaned_name,is_color_description,is_poetry,writing_form,writing_type
1,"['October Mist', 'Matching Colors', 'Different...",/en-us/paint-colors/color/1495/october-mist,October Mist 1495 | Benjamin Moore,Color,"Evocative of the stem of a flower, this gently...",650d633a3fb633324afcf6161c43826204085af7,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-21T22:48:45Z,2024-07-21T22:48:45Z,...,NaN,NaN,NaN,NaN,NaN,October Mist,yes,yes,poetry,color description
59,"['Huntington Green', 'Matching Colors', 'Diffe...",/en-us/paint-colors/color/406/huntington-green,Huntington Green 406 | Benjamin Moore,Color,"Rich and earthy, this yellow-green immediately...",0b5a1b6b87169b6ffcc10b13a062b0c4537cc009,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-06-29T17:52:09Z,2024-06-29T17:52:08Z,...,NaN,NaN,NaN,NaN,NaN,Huntington Green,yes,yes,poetry,color description
66,"['Calming Green', 'Matching Colors', 'Differen...",/en-us/paint-colors/color/605/calming-green,Calming Green 605 | Benjamin Moore,Color,"Buoyed by hints of blue, this spirited green h...",63e7d7bfe4f95a8e8b55ed0d95eacda8aa71a67e,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-06-29T17:37:58Z,2024-06-29T17:37:58Z,...,NaN,NaN,NaN,NaN,NaN,Calming Green,yes,yes,poetry,color description
110,"['White Rain', 'Matching Colors', 'Different S...",/en-us/paint-colors/color/708/white-rain,White Rain 708 | Benjamin Moore,Color,A soft blue-gray that cleanses like a cool sum...,007f32886e0cf05014463f514fa694684da9e43f,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-21T20:36:32Z,2024-07-21T20:36:32Z,...,NaN,NaN,NaN,NaN,NaN,White Rain,yes,yes,poetry,color description
139,"['Devonshire Green', 'Matching Colors', 'Diffe...",/en-us/paint-colors/color/1489/devonshire-green,Devonshire Green 1489 | Benjamin Moore,Color,An adaptable dark hue that can conjure rain-so...,9ca169b4eec105a603ab36fbfb597c90e11b058c,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T03:19:56Z,2024-07-22T03:19:55Z,...,NaN,NaN,NaN,NaN,NaN,Devonshire Green,yes,yes,poetry,color description
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3560,"['Porch Swing', 'Matching Colors', 'Different ...",/en-us/paint-colors/color/csp-750/porch-swing,Porch Swing CSP-750 | Benjamin Moore,Color,Lazy summer days sipping lemonade how quickly ...,a5177078bbc892da533502a9f2064f8403a06c35,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-31T04:54:25Z,2024-07-03T15:15:52Z,...,Living Room,Modern,NaN,NaN,NaN,Porch Swing,no,yes,poetry,poetry
3562,"['Nightfall', 'Matching Colors', 'Different Sh...",/en-us/paint-colors/color/1596/nightfall,Nightfall 1596 | Benjamin Moore,Color,A blackened gray infused with the serene mysti...,9b997ff94b893c733c9234601d68595cde51dbb4,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-21T23:32:49Z,2024-07-21T23:32:48Z,...,NaN,NaN,NaN,NaN,NaN,Nightfall,yes,yes,poetry,color description
3591,"['Vintage Taupe', 'Matching Colors', 'Differen...",/en-us/paint-colors/color/2110-70/vintage-taupe,Vintage Taupe 2110-70 | Benjamin Moore,Color,The tiniest drop of pink colors this hushed hue.,a7e1a9e05021720cb9e852d6d62257f14adff4fa,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-21T19:35:02Z,2024-07-21T19:35:02Z,...,NaN,NaN,NaN,NaN,NaN,Vintage Taupe,yes,yes,poetry,color description
3600,"['Ivory Tusk', 'Matching Colors', 'Similar Col...",/en-us/paint-colors/color/oc-91/ivory-tusk,Ivory Tusk OC-91 | Benjamin Moore,Colo

In [89]:
full_merged[full_merged['is_color_description'] == 'no']

,sections,page_path,title,type,description,external_id,image,url,published_at,updated_at,...,Project,Style,Value,Color Family,Tone,cleaned_name,is_color_description,is_poetry,writing_form,writing_type
164,"['Thayer Green', 'Matching Colors', 'Different...",/en-us/paint-colors/color/csp-825/thayer-green,Thayer Green CSP-825 | Benjamin Moore,Color,"In his crisp dress uniform, the new cadet stro...",c1d5f5891a6e7101e9abb020697500d0520c4bb9,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T04:53:25Z,2024-07-22T04:53:25Z,...,NaN,NaN,NaN,NaN,NaN,Thayer Green,no,no,prose,story
445,"['Parisian Red®', 'Matching Colors', 'Differen...",/en-us/paint-colors/color/csp-1170/parisian-red,Parisian Red CSP-1170 | Benjamin Moore,Color,"Entering the historic hotel, the elegant stair...",b94b3bccab54f208f277816d2c57433408e7eef8,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T03:36:15Z,2024-07-03T11:52:55Z,...,NaN,NaN,NaN,NaN,NaN,Parisian Red,no,no,prose,story
1057,"['St. John Blue', 'Matching Colors', 'Differen...",/en-us/paint-colors/color/csp-675/st-john-blue,St. John Blue CSP-675 | Benjamin Moore,Color,Foghorns sounded through the morning mist as t...,3136b231ccc5bdb166c5683e0a7e32cac3b6cc12,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-03T10:08:40Z,2024-07-03T10:08:40Z,...,NaN,NaN,NaN,NaN,NaN,St. John Blue,no,no,prose,story
1088,"['Gray Gardens', 'Matching Colors', 'Different...",/en-us/paint-colors/color/csp-55/gray-gardens,Gray Gardens CSP-55 | Benjamin Moore,Color,"Autumn is upon the old manse. Its romantic, ne...",3dc051df32fa0b086796b871a3ed61d84165c9b4,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T00:54:49Z,2024-07-22T00:54:49Z,...,NaN,NaN,NaN,NaN,NaN,Gray Gardens,no,yes,poetry,poetry
1189,"['Elderberry Wine', 'Matching Colors', 'Differ...",/en-us/paint-colors/color/csp-470/elderberry-wine,Elderberry Wine CSP-470 | Benjamin Moore,Color,"Take a deep drink of this rich, intoxicating w...",8691199e498d73da7aa549fc4a05b54c10172416,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-21T20:32:51Z,2024-07-20T15:36:20Z,...,NaN,NaN,NaN,NaN,NaN,Elderberry Wine,no,yes,poetry,poetry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3506,"['Knitted Cape', 'Matching Colors', 'Different...",/en-us/paint-colors/color/csp-965/knitted-cape,Knitted Cape CSP-965 | Benjamin Moore,Color,"Knit one, purl two… a lovingly crafted cape br...",a6b60f4b7467285342b4bbce6ef3c099eb19317a,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-30T20:38:24Z,2024-07-30T20:38:23Z,...,NaN,NaN,NaN,NaN,NaN,Knitted Cape,no,yes,poetry,story
3535,"['Make Believe', 'Matching Colors', 'Different...",/en-us/paint-colors/color/csp-1035/make-believe,Make Believe CSP-1035 | Benjamin Moore,Color,"The soft linen tablecloth, drying on the cloth...",a23a3b3f39a452c3591a1508539098e74614db99,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T02:59:13Z,2024-07-22T02:59:13Z,...,NaN,NaN,NaN,NaN,NaN,Make Believe,no,no,prose,story
3538,"['Lace Handkerchief', 'Matching Colors', 'Diff...",/en-us/paint-colors/color/csp-220/lace-handker...,Lace Handkerchief CSP-220 | Benjamin Moore,Color,She tucked the borrowed handkerchief into the ...,a030e2539eea2b5f09cf3e0d0c401cf68e9249d8,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-21T17:44:08Z,2024-07-21T17:44:07Z,...,NaN,NaN,NaN,NaN,NaN,Lace Handkerchief,no,no,prose,story
3558,"['Sea Glass', 'Matching Colors', 'Different Sh...",/en-us/paint-colors/color/csp-735/sea-glass,Sea Glass CSP-735 | Benjamin Moore,Color,"He bent down to pick up the smooth, 

In [90]:
full_merged[full_merged['writing_type'] == 'historically specific']

,sections,page_path,title,type,description,external_id,image,url,published_at,updated_at,...,Project,Style,Value,Color Family,Tone,cleaned_name,is_color_description,is_poetry,writing_form,writing_type
143,"['Timson Green', 'Matching Colors', 'Similar C...",/en-us/paint-colors/color/cw-470/timson-green,Timson Green CW-470 | Benjamin Moore,Color,Based on early 19th century paint evidence fou...,9edecf61acbce2727fbad1fcc9871d36a4ca6d8d,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-20T23:09:49Z,2024-07-20T23:09:49Z,...,NaN,NaN,NaN,NaN,NaN,Timson Green,yes,no,prose,historically specific


In [88]:
full_merged[full_merged['writing_type'] == 'story']

,sections,page_path,title,type,description,external_id,image,url,published_at,updated_at,...,Project,Style,Value,Color Family,Tone,cleaned_name,is_color_description,is_poetry,writing_form,writing_type
164,"['Thayer Green', 'Matching Colors', 'Different...",/en-us/paint-colors/color/csp-825/thayer-green,Thayer Green CSP-825 | Benjamin Moore,Color,"In his crisp dress uniform, the new cadet stro...",c1d5f5891a6e7101e9abb020697500d0520c4bb9,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T04:53:25Z,2024-07-22T04:53:25Z,...,NaN,NaN,NaN,NaN,NaN,Thayer Green,no,no,prose,story
445,"['Parisian Red®', 'Matching Colors', 'Differen...",/en-us/paint-colors/color/csp-1170/parisian-red,Parisian Red CSP-1170 | Benjamin Moore,Color,"Entering the historic hotel, the elegant stair...",b94b3bccab54f208f277816d2c57433408e7eef8,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T03:36:15Z,2024-07-03T11:52:55Z,...,NaN,NaN,NaN,NaN,NaN,Parisian Red,no,no,prose,story
1057,"['St. John Blue', 'Matching Colors', 'Differen...",/en-us/paint-colors/color/csp-675/st-john-blue,St. John Blue CSP-675 | Benjamin Moore,Color,Foghorns sounded through the morning mist as t...,3136b231ccc5bdb166c5683e0a7e32cac3b6cc12,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-03T10:08:40Z,2024-07-03T10:08:40Z,...,NaN,NaN,NaN,NaN,NaN,St. John Blue,no,no,prose,story
1324,"['Cinnabar', 'Matching Colors', 'Different Sha...",/en-us/paint-colors/color/csp-1165/cinnabar,Cinnabar CSP-1165 | Benjamin Moore,Color,Warm cinnamon sticks and cloves simmered away ...,60d762e7e065916da8214340777974a7997eb007,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-31T02:09:19Z,2024-07-03T15:45:20Z,...,NaN,NaN,NaN,NaN,NaN,Cinnabar,no,no,prose,story
1333,"['Cherry Burst', 'Matching Colors', 'Different...",/en-us/paint-colors/color/csp-1200/cherry-burst,Cherry Burst CSP-1200 | Benjamin Moore,Color,"Tart and sweet at the same time, like a homema...",37351a59956d26677ef9acfa0ff63b30298a4689,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-30T20:41:17Z,2024-07-03T16:48:54Z,...,NaN,NaN,NaN,NaN,NaN,Cherry Burst,no,no,prose,story
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3472,"['Iced Coffee', 'Matching Colors', 'Different ...",/en-us/paint-colors/color/csp-985/iced-coffee,Iced Coffee CSP-985 | Benjamin Moore,Color,"Take one cup of full-brewed espresso, add heav...",a7f2446cb07f1014f9450acb570ba27e05c8709d,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-30T14:47:02Z,2024-07-03T16:42:09Z,...,NaN,NaN,NaN,NaN,NaN,Iced Coffee,no,no,prose,story
3506,"['Knitted Cape', 'Matching Colors', 'Different...",/en-us/paint-colors/color/csp-965/knitted-cape,Knitted Cape CSP-965 | Benjamin Moore,Color,"Knit one, purl two… a lovingly crafted cape br...",a6b60f4b7467285342b4bbce6ef3c099eb19317a,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-30T20:38:24Z,2024-07-30T20:38:23Z,...,NaN,NaN,NaN,NaN,NaN,Knitted Cape,no,yes,poetry,story
3535,"['Make Believe', 'Matching Colors', 'Different...",/en-us/paint-colors/color/csp-1035/make-believe,Make Believe CSP-1035 | Benjamin Moore,Color,"The soft linen tablecloth, drying on the cloth...",a23a3b3f39a452c3591a1508539098e74614db99,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T02:59:13Z,2024-07-22T02:59:13Z,...,NaN,NaN,NaN,NaN,NaN,Make Believe,no,no,prose,story
3538,"['Lace Handkerchief', 'Matching Colors', 'Diff...",/en-us/paint-colors/color/csp-220/lace-handker...,Lace Handkerchief CSP-220 | Benjamin Moore,Color,She tucked the borrowed handkerchief into the ...,a030e2539e

In [91]:
# # Filter the DataFrame for rows where writing_type = story or poetry
values_to_filter = ['poetry', 'story']

story_poetry_df = full_merged[full_merged['writing_type'].isin(values_to_filter)]
story_poetry_df

,sections,page_path,title,type,description,external_id,image,url,published_at,updated_at,...,Project,Style,Value,Color Family,Tone,cleaned_name,is_color_description,is_poetry,writing_form,writing_type
164,"['Thayer Green', 'Matching Colors', 'Different...",/en-us/paint-colors/color/csp-825/thayer-green,Thayer Green CSP-825 | Benjamin Moore,Color,"In his crisp dress uniform, the new cadet stro...",c1d5f5891a6e7101e9abb020697500d0520c4bb9,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T04:53:25Z,2024-07-22T04:53:25Z,...,NaN,NaN,NaN,NaN,NaN,Thayer Green,no,no,prose,story
445,"['Parisian Red®', 'Matching Colors', 'Differen...",/en-us/paint-colors/color/csp-1170/parisian-red,Parisian Red CSP-1170 | Benjamin Moore,Color,"Entering the historic hotel, the elegant stair...",b94b3bccab54f208f277816d2c57433408e7eef8,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T03:36:15Z,2024-07-03T11:52:55Z,...,NaN,NaN,NaN,NaN,NaN,Parisian Red,no,no,prose,story
1054,"['Fair Isle Blue', 'Matching Colors', 'Differe...",/en-us/paint-colors/color/csp-715/fair-isle-blue,Fair Isle Blue CSP-715 | Benjamin Moore,Color,"Rowing the deep blue, almost green sea, the fi...",66e30fed7f9b04b5361da798444f085b792edcab,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-21T20:28:12Z,2024-07-21T20:28:12Z,...,NaN,NaN,NaN,NaN,NaN,Fair Isle Blue,yes,yes,poetry,poetry
1057,"['St. John Blue', 'Matching Colors', 'Differen...",/en-us/paint-colors/color/csp-675/st-john-blue,St. John Blue CSP-675 | Benjamin Moore,Color,Foghorns sounded through the morning mist as t...,3136b231ccc5bdb166c5683e0a7e32cac3b6cc12,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-03T10:08:40Z,2024-07-03T10:08:40Z,...,NaN,NaN,NaN,NaN,NaN,St. John Blue,no,no,prose,story
1088,"['Gray Gardens', 'Matching Colors', 'Different...",/en-us/paint-colors/color/csp-55/gray-gardens,Gray Gardens CSP-55 | Benjamin Moore,Color,"Autumn is upon the old manse. Its romantic, ne...",3dc051df32fa0b086796b871a3ed61d84165c9b4,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T00:54:49Z,2024-07-22T00:54:49Z,...,NaN,NaN,NaN,NaN,NaN,Gray Gardens,no,yes,poetry,poetry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3506,"['Knitted Cape', 'Matching Colors', 'Different...",/en-us/paint-colors/color/csp-965/knitted-cape,Knitted Cape CSP-965 | Benjamin Moore,Color,"Knit one, purl two… a lovingly crafted cape br...",a6b60f4b7467285342b4bbce6ef3c099eb19317a,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-30T20:38:24Z,2024-07-30T20:38:23Z,...,NaN,NaN,NaN,NaN,NaN,Knitted Cape,no,yes,poetry,story
3535,"['Make Believe', 'Matching Colors', 'Different...",/en-us/paint-colors/color/csp-1035/make-believe,Make Believe CSP-1035 | Benjamin Moore,Color,"The soft linen tablecloth, drying on the cloth...",a23a3b3f39a452c3591a1508539098e74614db99,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T02:59:13Z,2024-07-22T02:59:13Z,...,NaN,NaN,NaN,NaN,NaN,Make Believe,no,no,prose,story
3538,"['Lace Handkerchief', 'Matching Colors', 'Diff...",/en-us/paint-colors/color/csp-220/lace-handker...,Lace Handkerchief CSP-220 | Benjamin Moore,Color,She tucked the borrowed handkerchief into the ...,a030e2539eea2b5f09cf3e0d0c401cf68e9249d8,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-21T17:44:08Z,2024-07-21T17:44:07Z,...,NaN,NaN,NaN,NaN,NaN,Lace Handkerchief,no,no,prose,story
3558,"['Sea Glass', 'Matching Colors', 'Different Sh...",/en-us/paint-colors/color/csp-735/sea-glass,Sea Glass CSP-735 | Benjamin Moore,Color,"He bent down to pick up the smooth, pa

In [92]:
story_poetry_df.to_csv('story_poetry.csv')

## I then went through that csv and added my own classification column. Then read in that csv:

In [93]:
manual_labels = pd.read_csv('story_poetry_manual.csv')
manual_labels

,Unnamed: 0,sections,page_path,title,type,description,external_id,image,url,published_at,...,Style,Value,Color Family,Tone,cleaned_name,is_color_description,is_poetry,writing_form,writing_type,AK_writing_type
0,164,"['Thayer Green', 'Matching Colors', 'Different...",/en-us/paint-colors/color/csp-825/thayer-green,Thayer Green CSP-825 | Benjamin Moore,Color,"In his crisp dress uniform, the new cadet stro...",c1d5f5891a6e7101e9abb020697500d0520c4bb9,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T04:53:25Z,...,NaN,NaN,NaN,NaN,Thayer Green,no,no,prose,story,story
1,445,"['Parisian Red®', 'Matching Colors', 'Differen...",/en-us/paint-colors/color/csp-1170/parisian-red,Parisian Red CSP-1170 | Benjamin Moore,Color,"Entering the historic hotel, the elegant stair...",b94b3bccab54f208f277816d2c57433408e7eef8,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T03:36:15Z,...,NaN,NaN,NaN,NaN,Parisian Red,no,no,prose,story,story
2,1054,"['Fair Isle Blue', 'Matching Colors', 'Differe...",/en-us/paint-colors/color/csp-715/fair-isle-blue,Fair Isle Blue CSP-715 | Benjamin Moore,Color,"Rowing the deep blue, almost green sea, the fi...",66e30fed7f9b04b5361da798444f085b792edcab,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-21T20:28:12Z,...,NaN,NaN,NaN,NaN,Fair Isle Blue,yes,yes,poetry,poetry,story
3,1057,"['St. John Blue', 'Matching Colors', 'Differen...",/en-us/paint-colors/color/csp-675/st-john-blue,St. John Blue CSP-675 | Benjamin Moore,Color,Foghorns sounded through the morning mist as t...,3136b231ccc5bdb166c5683e0a7e32cac3b6cc12,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-03T10:08:40Z,...,NaN,NaN,NaN,NaN,St. John Blue,no,no,prose,story,story
4,1088,"['Gray Gardens', 'Matching Colors', 'Different...",/en-us/paint-colors/color/csp-55/gray-gardens,Gray Gardens CSP-55 | Benjamin Moore,Color,"Autumn is upon the old manse. Its romantic, ne...",3dc051df32fa0b086796b871a3ed61d84165c9b4,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T00:54:49Z,...,NaN,NaN,NaN,NaN,Gray Gardens,no,yes,poetry,poetry,story
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,3506,"['Knitted Cape', 'Matching Colors', 'Different...",/en-us/paint-colors/color/csp-965/knitted-cape,Knitted Cape CSP-965 | Benjamin Moore,Color,"Knit one, purl two… a lovingly crafted cape br...",a6b60f4b7467285342b4bbce6ef3c099eb19317a,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-30T20:38:24Z,...,NaN,NaN,NaN,NaN,Knitted Cape,no,yes,poetry,story,story
99,3535,"['Make Believe', 'Matching Colors', 'Different...",/en-us/paint-colors/color/csp-1035/make-believe,Make Believe CSP-1035 | Benjamin Moore,Color,"The soft linen tablecloth, drying on the cloth...",a23a3b3f39a452c3591a1508539098e74614db99,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T02:59:13Z,...,NaN,NaN,NaN,NaN,Make Believe,no,no,prose,story,story
100,3538,"['Lace Handkerchief', 'Matching Colors', 'Diff...",/en-us/paint-colors/color/csp-220/lace-handker...,Lace Handkerchief CSP-220 | Benjamin Moore,Color,She tucked the borrowed handkerchief into the ...,a030e2539eea2b5f09cf3e0d0c401cf68e9249d8,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-21T17:44:08Z,...,NaN,NaN,NaN,NaN,Lace Handkerchief,no,no,prose,story,story
101,3558,"['Sea Glass', 'Matching Colors', 'Different Sh...",/en-us/paint-colors/color/csp-735/sea-glass,Sea Glass CSP-735 | Benjamin Moore,Color,"He bent down to pick up the smooth, pale fragm...",8cb484b35f4ffe3e0594c5397da65947accc200b,https://media.benjaminmoore.com/WebServices/pr...,https://www.benj

In [94]:
manual_labels['AK_writing_type'].value_counts()

AK_writing_type
story              95
not quite story     8
Name: count, dtype: int64

In [95]:
# The description column in this database is what I want to feed into chatgpt to write a story. 95 sentences.
manual_labels[manual_labels['AK_writing_type'] == 'story']

,Unnamed: 0,sections,page_path,title,type,description,external_id,image,url,published_at,...,Style,Value,Color Family,Tone,cleaned_name,is_color_description,is_poetry,writing_form,writing_type,AK_writing_type
0,164,"['Thayer Green', 'Matching Colors', 'Different...",/en-us/paint-colors/color/csp-825/thayer-green,Thayer Green CSP-825 | Benjamin Moore,Color,"In his crisp dress uniform, the new cadet stro...",c1d5f5891a6e7101e9abb020697500d0520c4bb9,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T04:53:25Z,...,NaN,NaN,NaN,NaN,Thayer Green,no,no,prose,story,story
1,445,"['Parisian Red®', 'Matching Colors', 'Differen...",/en-us/paint-colors/color/csp-1170/parisian-red,Parisian Red CSP-1170 | Benjamin Moore,Color,"Entering the historic hotel, the elegant stair...",b94b3bccab54f208f277816d2c57433408e7eef8,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T03:36:15Z,...,NaN,NaN,NaN,NaN,Parisian Red,no,no,prose,story,story
2,1054,"['Fair Isle Blue', 'Matching Colors', 'Differe...",/en-us/paint-colors/color/csp-715/fair-isle-blue,Fair Isle Blue CSP-715 | Benjamin Moore,Color,"Rowing the deep blue, almost green sea, the fi...",66e30fed7f9b04b5361da798444f085b792edcab,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-21T20:28:12Z,...,NaN,NaN,NaN,NaN,Fair Isle Blue,yes,yes,poetry,poetry,story
3,1057,"['St. John Blue', 'Matching Colors', 'Differen...",/en-us/paint-colors/color/csp-675/st-john-blue,St. John Blue CSP-675 | Benjamin Moore,Color,Foghorns sounded through the morning mist as t...,3136b231ccc5bdb166c5683e0a7e32cac3b6cc12,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-03T10:08:40Z,...,NaN,NaN,NaN,NaN,St. John Blue,no,no,prose,story,story
4,1088,"['Gray Gardens', 'Matching Colors', 'Different...",/en-us/paint-colors/color/csp-55/gray-gardens,Gray Gardens CSP-55 | Benjamin Moore,Color,"Autumn is upon the old manse. Its romantic, ne...",3dc051df32fa0b086796b871a3ed61d84165c9b4,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T00:54:49Z,...,NaN,NaN,NaN,NaN,Gray Gardens,no,yes,poetry,poetry,story
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
98,3506,"['Knitted Cape', 'Matching Colors', 'Different...",/en-us/paint-colors/color/csp-965/knitted-cape,Knitted Cape CSP-965 | Benjamin Moore,Color,"Knit one, purl two… a lovingly crafted cape br...",a6b60f4b7467285342b4bbce6ef3c099eb19317a,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-30T20:38:24Z,...,NaN,NaN,NaN,NaN,Knitted Cape,no,yes,poetry,story,story
99,3535,"['Make Believe', 'Matching Colors', 'Different...",/en-us/paint-colors/color/csp-1035/make-believe,Make Believe CSP-1035 | Benjamin Moore,Color,"The soft linen tablecloth, drying on the cloth...",a23a3b3f39a452c3591a1508539098e74614db99,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T02:59:13Z,...,NaN,NaN,NaN,NaN,Make Believe,no,no,prose,story,story
100,3538,"['Lace Handkerchief', 'Matching Colors', 'Diff...",/en-us/paint-colors/color/csp-220/lace-handker...,Lace Handkerchief CSP-220 | Benjamin Moore,Color,She tucked the borrowed handkerchief into the ...,a030e2539eea2b5f09cf3e0d0c401cf68e9249d8,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-21T17:44:08Z,...,NaN,NaN,NaN,NaN,Lace Handkerchief,no,no,prose,story,story
101,3558,"['Sea Glass', 'Matching Colors', 'Different Sh...",/en-us/paint-colors/color/csp-735/sea-glass,Sea Glass CSP-735 | Benjamin Moore,Color,"He bent down to pick up the smooth, pale fragm...",8cb484b35f4ffe3e0594c5397da65947accc200b,https://media.benjaminmoore.com/WebServices/pr...,https://www.benj

# Analyzing data for graphics

In [122]:
full_merged['is_color_description'].value_counts(normalize=True)

is_color_description
yes    0.971835
no     0.028165
Name: proportion, dtype: float64

In [123]:
full_merged['is_poetry'].value_counts(normalize=True)

is_poetry
no     0.934372
yes    0.065628
Name: proportion, dtype: float64

In [120]:
full_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3657 entries, 0 to 3656
Data columns (total 45 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   sections              3657 non-null   object 
 1   page_path             3657 non-null   object 
 2   title                 3657 non-null   object 
 3   type                  3657 non-null   object 
 4   description           3657 non-null   object 
 5   external_id           3657 non-null   object 
 6   image                 3657 non-null   object 
 7   url                   3657 non-null   object 
 8   published_at          3657 non-null   object 
 9   updated_at            3657 non-null   object 
 10  cdp                   400 non-null    object 
 11  popularity            3657 non-null   int64  
 12  info                  0 non-null      float64
 13  _index                1183 non-null   object 
 14  _type                 1183 non-null   object 
 15  _score               

In [6]:
full_merged['PrimaryColorFamily'].value_counts()

NameError: name 'full_merged' is not defined

In [99]:
full_merged['Color Family'].value_counts()

Color Family
['Orange', 'Red']      2
Yellow                 1
['Green', 'Yellow']    1
Green                  1
Blue                   1
Name: count, dtype: int64

In [100]:
# neither of these have enough values in them to be useful, need to join.
full_merged.groupby(['PrimaryColorFamily'])['is_color_description'].value_counts()

PrimaryColorFamily  is_color_description
Neutral             yes                     702
                    no                       34
Orange              yes                     345
                    no                        5
Red                 yes                     394
                    no                        7
White               yes                     234
                    no                        7
Yellow              yes                     309
                    no                        1
Name: count, dtype: int64

In [110]:
len(full_merged[full_merged['is_color_description'] == 'no'][full_merged['is_poetry'] == 'yes'])
# poems but not descriptions

/var/folders/v6/gl0vmz3x7_35wvmcn3_6vxf40000gn/T/ipykernel_9934/1667387875.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(full_merged[full_merged['is_color_description'] == 'no'][full_merged['is_poetry'] == 'yes'])


44

In [112]:
len(full_merged[full_merged['is_color_description'] == 'no'][full_merged['is_poetry'] == 'no'])
# neither poems nor descriptions

/var/folders/v6/gl0vmz3x7_35wvmcn3_6vxf40000gn/T/ipykernel_9934/2459987621.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(full_merged[full_merged['is_color_description'] == 'no'][full_merged['is_poetry'] == 'no'])


59

In [111]:
len(full_merged[full_merged['is_color_description'] == 'yes'][full_merged['is_poetry'] == 'yes'])
# poetic descriptions

/var/folders/v6/gl0vmz3x7_35wvmcn3_6vxf40000gn/T/ipykernel_9934/1966080520.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(full_merged[full_merged['is_color_description'] == 'yes'][full_merged['is_poetry'] == 'yes'])


196

In [125]:
len(full_merged[full_merged['is_color_description'] == 'yes'][full_merged['is_poetry'] == 'no'])
# descriptions not poems

/var/folders/v6/gl0vmz3x7_35wvmcn3_6vxf40000gn/T/ipykernel_9934/2850619046.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  len(full_merged[full_merged['is_color_description'] == 'yes'][full_merged['is_poetry'] == 'no'])


3358

In [119]:
not_color_not_poem = pd.DataFrame(full_merged[full_merged['is_color_description'] == 'no'][full_merged['is_poetry'] == 'no']['description'])
print(not_color_not_poem['description'])

164     In his crisp dress uniform, the new cadet stro...
445     Entering the historic hotel, the elegant stair...
1057    Foghorns sounded through the morning mist as t...
1209    Robust, full-flavored heat lies within the dee...
1324    Warm cinnamon sticks and cloves simmered away ...
1333    Tart and sweet at the same time, like a homema...
1388    The delicate stitches withstood nearly a centu...
1409    Savoring the fruity tea and the pink tapioca b...
1443    The earthy tiles cooled her feet as she entere...
1656    A rugged trail lay before them as they began t...
1686    The intoxicating aromas of the sweet tropical ...
1765    The ancient floor in the Florentine villa has ...
1792    As they left the dance, he placed the soft pas...
1803    Newly polished, my favorite boots were ready f...
1852    A roasted Indian delight, full of paprika, cor...
1982    Delicate and soft, the exquisite powder in the...
1990    Her knapsack was softened and well worn from t...
2004    The pe

/var/folders/v6/gl0vmz3x7_35wvmcn3_6vxf40000gn/T/ipykernel_9934/2462105917.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  not_color_not_poem = pd.DataFrame(full_merged[full_merged['is_color_description'] == 'no'][full_merged['is_poetry'] == 'no']['description'])


In [126]:
not_color_not_poem.to_csv('not_color_not_poem.csv')

In [7]:
family_df = pd.read_csv('family_pages.csv')
family_df

,color_name,color_code,color_fam
0,White Opulence,OC-69,red
1,Alabaster,OC-129,red
2,Gardenia,AF-10,red
3,Pink Damask,OC-72,red
4,Sand Dollar,OC-71,red
...,...,...,...
3584,White Diamond,OC-61,white
3585,Harwood Putty,CW-5,white
3586,Capitol White,CW-10,white
3587,Parish White,CW-15,white


In [8]:
family_df['color_name'].value_counts(ascending = False)

color_name
Shooting Star         5
Country Comfort       5
Nacho Cheese          5
Oxford Gold           5
Sunflower Fields      5
                     ..
Night Mist            1
Black Forest Green    1
Raleigh Sorrel        1
York Gray             1
Florida Keys Blue     1
Name: count, Length: 2796, dtype: int64

In [134]:
new_full_merged = full_merged.merge(family_df[['color_name', 'color_fam']], left_on = 'color_name', right_on = 'color_name', how = 'left')
new_full_merged

,sections,page_path,title,type,description,external_id,image,url,published_at,updated_at,...,Style,Value,Color Family,Tone,cleaned_name,is_color_description,is_poetry,writing_form,writing_type,color_fam
0,"['Hunter Green', 'Matching Colors', 'Different...",/en-us/paint-colors/color/2041-10/hunter-green,Hunter Green 2041-10 | Benjamin Moore,Color,A timeless dark green full of depth and elegance.,5bbc15c9d8fa62ad213913ec73d45dd3fd96927f,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T00:32:36Z,2024-07-22T00:32:36Z,...,NaN,NaN,NaN,NaN,Hunter Green,yes,no,prose,color description,green
1,"['October Mist', 'Matching Colors', 'Different...",/en-us/paint-colors/color/1495/october-mist,October Mist 1495 | Benjamin Moore,Color,"Evocative of the stem of a flower, this gently...",650d633a3fb633324afcf6161c43826204085af7,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-21T22:48:45Z,2024-07-21T22:48:45Z,...,NaN,NaN,NaN,NaN,October Mist,yes,yes,poetry,color description,green
2,"['Hollingsworth Green', 'Matching Colors', 'Di...",/en-us/paint-colors/color/hc-141/hollingsworth...,Hollingsworth Green HC-141 | Benjamin Moore,Color,"A feather-light combination of green, blue and...",c3b74b8ba97769e1a4ef88f2ec55bdeef8b02d1f,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-21T22:40:18Z,2024-07-21T22:40:18Z,...,NaN,NaN,NaN,NaN,Hollingsworth Green,yes,no,prose,color description,green
3,"['Guilford Green', 'Matching Colors', 'Differe...",/en-us/paint-colors/color/hc-116/guilford-green,Guilford Green HC-116 | Benjamin Moore,Color,A gentle green hue with pleasing warm undertones.,30c65a442a898b35d8c2433b1f7d565b4fcef0ec,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T00:15:41Z,2024-07-22T00:15:40Z,...,NaN,NaN,NaN,NaN,Guilford Green,yes,no,prose,color description,green
4,"['Soft Fern', 'Matching Colors', 'Different Sh...",/en-us/paint-colors/color/2144-40/soft-fern,Soft Fern 2144-40 | Benjamin Moore,Color,A pleasing pale green misted with gray tones.,1814c4578a537d1e3d02b407e92c0b11e06b2109,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-21T21:00:09Z,2024-07-21T21:00:08Z,...,NaN,NaN,NaN,NaN,Soft Fern,yes,no,prose,color description,green
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4073,"[""Marilyn's Dress"", 'Matching Colors', 'Differ...",/en-us/paint-colors/color/2125-60/marilyns-dress,Marilyn's Dress 2125-60 | Benjamin Moore,Color,"Much like its namesake, this pale blue-gray is...",131ede0694dd47eaf161b4579d04e4e682b5209f,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-21T21:22:05Z,2024-07-21T21:22:05Z,...,NaN,NaN,NaN,NaN,Marilyn's Dress,yes,no,prose,color description,white
4074,"['Tundra', 'Matching Colors', 'Different Shade...",/en-us/paint-colors/color/2133-70/tundra,Tundra 2133-70 | Benjamin Moore,Color,Trace amounts of gray and violet come through ...,6e037ee23fe11ec6c2979db140c6f6ef7f27a803,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-21T09:00:21Z,2024-07-21T09:00:21Z,...,NaN,NaN,NaN,NaN,Tundra,yes,no,prose,color description,white
4075,"['Winter Snow', 'Matching Colors', 'Similar Co...",/en-us/paint-colors/color/oc-63/winter-snow,Winter Snow OC-63 | Benjamin Moore,Color,Clean blue undertones define this optic white.,ae5eb68474567462c0ad84186570f81e9abc7f36,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-22T01:31:20Z,2024-07-22T01:31:20Z,...,NaN,NaN,NaN,NaN,Winter Snow,yes,no,prose,color description,white
4076,"[""Baby's Breath"", 'Matching Colors', 'Differen...",/en-us/paint-colors/color/873/babys-breath,Baby's Breath 873 | Ben

In [158]:
new_full_merged[new_full_merged['writing_type'] == 'historically specific']

,sections,page_path,title,type,description,external_id,image,url,published_at,updated_at,...,Style,Value,Color Family,Tone,cleaned_name,is_color_description,is_poetry,writing_form,writing_type,color_fam
148,"['Timson Green', 'Matching Colors', 'Similar C...",/en-us/paint-colors/color/cw-470/timson-green,Timson Green CW-470 | Benjamin Moore,Color,Based on early 19th century paint evidence fou...,9edecf61acbce2727fbad1fcc9871d36a4ca6d8d,https://media.benjaminmoore.com/WebServices/pr...,https://www.benjaminmoore.com/en-us/paint-colo...,2024-07-20T23:09:49Z,2024-07-20T23:09:49Z,...,NaN,NaN,NaN,NaN,Timson Green,yes,no,prose,historically specific,green


In [136]:
new_full_merged['cleaned_name'].value_counts(ascending = False)

cleaned_name
Camouflage          2
Tavern Charcoal     2
Bone Black          2
Mopboard Black      2
Ambler Slate        2
                   ..
Desert Green        1
Green Wave          1
Adirondack Green    1
Peale Green         1
Saybrook Sage       1
Name: count, Length: 3657, dtype: int64

In [137]:
new_full_merged = new_full_merged.drop_duplicates('cleaned_name')

In [139]:
new_full_merged.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3657 entries, 0 to 4077
Data columns (total 46 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   sections              3657 non-null   object 
 1   page_path             3657 non-null   object 
 2   title                 3657 non-null   object 
 3   type                  3657 non-null   object 
 4   description           3657 non-null   object 
 5   external_id           3657 non-null   object 
 6   image                 3657 non-null   object 
 7   url                   3657 non-null   object 
 8   published_at          3657 non-null   object 
 9   updated_at            3657 non-null   object 
 10  cdp                   400 non-null    object 
 11  popularity            3657 non-null   int64  
 12  info                  0 non-null      float64
 13  _index                1183 non-null   object 
 14  _type                 1183 non-null   object 
 15  _score                1183

In [157]:
3657-3571

86

In [140]:
new_full_merged.groupby(['color_fam'])['writing_type'].value_counts(normalize=True, ascending=False)

color_fam  writing_type         
blue       color description        0.985258
           story                    0.009828
           poetry                   0.004914
gray       color description        0.933492
           story                    0.040380
           poetry                   0.026128
green      color description        0.978998
           story                    0.012924
           poetry                   0.006462
           historically specific    0.001616
neutral    color description        0.934783
           story                    0.054348
           poetry                   0.010870
orange     color description        0.980658
           story                    0.017408
           poetry                   0.001934
purple     color description        1.000000
red        color description        0.974832
           story                    0.015101
           poetry                   0.010067
white      color description        1.000000
yellow     color descr

In [144]:
fam_df = pd.DataFrame(new_full_merged.groupby(['color_fam'])['writing_type'].value_counts().reset_index())
fam_df

,color_fam,writing_type,count
0,blue,color description,401
1,blue,story,4
2,blue,poetry,2
3,gray,color description,393
4,gray,story,17
5,gray,poetry,11
6,green,color description,606
7,green,story,8
8,green,poetry,4
9,green,historically specific,1


In [155]:
fam_totals = fam_df.pivot(
        index="color_fam",
        columns="writing_type",
        values="number"
    )
fam_totals

writing_type,color description,historically specific,poetry,story
color_fam,,,,
blue,401.0,NaN,2.0,4.0
gray,393.0,NaN,11.0,17.0
green,606.0,1.0,4.0,8.0
neutral,344.0,NaN,4.0,20.0
orange,507.0,NaN,1.0,9.0
purple,232.0,NaN,NaN,NaN
red,581.0,NaN,6.0,9.0
white,172.0,NaN,NaN,NaN
yellow,235.0,NaN,3.0,1.0


In [156]:
fam_totals.to_csv('fam_totals.csv')